### test model

In [2]:
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from peft import PeftConfig, PeftModel, set_peft_model_state_dict
import cv2
from PIL import Image
import numpy as np

lora_path = "./post_model/AdaLora/epoch10"
control_path = "../autodl-tmp/models--lllyasviel--control_v11p_sd15_canny/snapshots/115a470d547982438f70198e353a921996e2e819"
SD_path = "../autodl-tmp/runwayml--stable-diffusion/snapshots/f03de327dd89b501a01da37fc5240cf4fdba85a1"

# 加载 ControlNet 和 Stable Diffusion
controlnet = ControlNetModel.from_pretrained(control_path)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    SD_path, controlnet=controlnet
).to("cuda")
# pipe.load_lora_weights(lora_path)
fine_model = PeftModel.from_pretrained(pipe.unet, lora_path)
set_peft_model_state_dict(pipe.unet, fine_model.state_dict())
pipe.unet = fine_model.model
pipe.safety_checker = None  # 禁用安全检查器

Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00, 11.24it/s]


In [3]:
import os

# 输入文件夹和输出文件夹路径
input_folder = "./human"
output_folder = "./generated_images/AdaLoRA_img"

# 创建输出文件夹（如果不存在）
os.makedirs(output_folder, exist_ok=True)

# 遍历 human 文件夹下的所有图片
for filename in os.listdir(input_folder):
    # 确保只处理图片文件
    if filename.endswith((".jpg", ".png", ".jpeg")):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, f"generated_{filename}")

        # 加载并预处理输入图像
        input_image = cv2.imread(input_path)
        gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, threshold1=100, threshold2=200)
        edges_image = Image.fromarray(edges)

        # 生成年画风格的图像
        prompt = "a traditional Chinese New Year painting style of a warrior figure, vibrant colors, detailed lines"
        generated_image = pipe(
            prompt, 
            image=edges_image, 
            num_inference_steps=120
        ).images[0]

        # 保存生成结果
        generated_image.save(output_path)
        print(f"Generated image saved to: {output_path}")

print("Processing complete. All images have been generated.")

100%|██████████| 120/120 [00:11<00:00, 10.42it/s]


Generated image saved to: ./AdaLoRA_img/generated_human_1.jpg


100%|██████████| 120/120 [01:16<00:00,  1.58it/s]


Generated image saved to: ./AdaLoRA_img/generated_human_2.png


100%|██████████| 120/120 [00:06<00:00, 18.44it/s]


...

Generated image saved to: ./AdaLoRA_img/generated_human_4.jpg


100%|██████████| 120/120 [02:45<00:00,  1.38s/it]


Generated image saved to: ./AdaLoRA_img/generated_human_5.jpg


100%|██████████| 120/120 [01:07<00:00,  1.78it/s]


Generated image saved to: ./AdaLoRA_img/generated_human_6.jpg
Processing complete. All images have been generated.
